In [1]:
import pandas as pd
import cv2
import numpy as np

In [4]:
folder='/udacity_images/'

In [5]:
df=pd.read_csv(folder+'driving_log.csv',\
            names=['center','left','right','measurement','x2','x3','x4'])

In [9]:
fname

'/Users/p.tarasov/Desktop/udacity_images/IMG/center_2017_04_08_20_41_33_635.jpg'

In [10]:
images=np.empty((len(df),160,320,3), dtype=np.uint8)
for i,fname in enumerate(df['center'].values):
    images[i]=cv2.imread(fname.replace('/Users/p.tarasov/Desktop/udacity_images/',folder))

In [24]:
from keras.models import Sequential
from keras.layers import Flatten,Dense, Convolution2D, Activation, MaxPooling2D, Lambda,Conv2D,Dropout

In [25]:
model=Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))

#model.add(Convolution2D(20, 5, 5, border_mode="same",
#input_shape=(160,320,3)))

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(160,320,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

In [26]:
model.compile(loss='mse',optimizer='adam')

In [ ]:
model.fit(images, df['measurement'].values, validation_split=0.2, shuffle=True, epochs=7)

Train on 5911 samples, validate on 1478 samples
Epoch 1/7
5911/5911 [==============================] - 45s - loss: 8.3647 - val_loss: 0.0132

In [20]:
model.save('model_cnn.h5')